In [ ]:
import itertools as it
import operator
import re
from collections import Counter
from pathlib import Path

import duckdb
import gfapy
import holoviews as hv
import ibis
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import pyabpoa
import pyarrow as pa
import pyarrow.compute as pc
import pyarrow.dataset as ds
import pyfastx
import pysam
import spoa
from pyarrow import csv
from tqdm.auto import tqdm, trange

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.sequencing.io as sio

In [ ]:
hv.extension("bokeh")

In [ ]:
%load_ext pyinstrument
import line_profiler
import pyinstrument

%load_ext line_profiler

# Config

In [ ]:
data_dir = Path(
    "/home/jqs1/scratch/jqs1/sequencing/230930_alignment_test/230707_repressilators/"
)

In [ ]:
arrow_ds = ds.dataset(list(data_dir.glob("*.arrow")), format="arrow")
arrow_ds2 = ds.dataset(list(data_dir.glob("*.arrow"))[:1], format="arrow")
parquet_ds = ds.dataset(list(data_dir.glob("*.parquet")), format="parquet")

# Group by path

## Setup

In [ ]:
gfa = gfapy.Gfa.from_file(
    "/home/jqs1/scratch/jqs1/sequencing/230930_alignment_test/barcode.gfa"
)

In [ ]:
bc_segments_oriented = [
    [f"{o}{s}" for s in gfa.segment_names if s.startswith("BC:")] for o in "><"
]

In [ ]:
bc_segments = [
    f"{o}{s}" for s in gfa.segment_names if s.startswith("BC:BIT") for o in "><"
]

In [ ]:
reverse_path_mapping = {
    f"{o[0]}{s}": f"{o[1]}{s}" for s in gfa.segment_names for o in ["<>", "><"]
}

## Polars

In [ ]:
df = pl.scan_pyarrow_dataset(arrow_ds2)
with pl.StringCache():
    res = (
        df.select(pl.col("path"))
        .limit(10)
        .with_columns(
            pl.col("path")
            .list.set_intersection(bc_segments)
            .cast(pl.List(pl.Categorical))
            .alias("path_subset"),
        )
        .with_columns(
            pl.col("path_subset")
            .list.reverse()
            .list.eval(pl.element().map_dict(reverse_path_mapping))
            .alias("path_subset_reversed"),
            (
                pl.col("path_subset")
                .list.set_intersection(bc_segments_oriented[0])
                .list.len()
                > 0
            ).alias("is_forward"),
            (
                pl.col("path_subset")
                .list.set_intersection(bc_segments_oriented[1])
                .list.len()
                > 0
            ).alias("is_reverse"),
        )
        .with_columns(
            pl.when(pl.col("is_forward") & pl.col("is_reverse").not_())
            .then(True)
            .when(pl.col("is_forward").not_() & pl.col("is_reverse"))
            .then(pl.lit(False))
            .otherwise(pl.lit(None))
            .alias("orientation"),  # how does this compare to strand?
        )
        .with_columns(
            pl.when(pl.col("orientation") == True)
            .then(pl.col("path_subset"))
            .when(pl.col("orientation") == False)
            .then(pl.col("path_subset_reversed"))
            .otherwise(pl.col("path_subset_reversed"))
            # pl.when(pl.col("orientation") == True)
            # .then(pl.col("path_subset"))
            # .then(pl.lit(1))
            # .when(pl.col("orientation") == False)
            # .then(pl.col("path_subset_reversed"))
            # .then(pl.lit(2))
            # .otherwise(pl.lit(0))
            # .otherwise(pl.col("path_subset_reversed"))
            .alias("path_subset_normalized")
        )
    ).collect()
res

In [ ]:
arrow_ds2.schema

In [ ]:
df = pl.scan_pyarrow_dataset(arrow_ds2)
with pl.StringCache():
    res = (
        df.limit(10)
        .with_columns(
            pl.col("path")
            .list.set_intersection(bc_segments)
            .cast(pl.List(pl.Categorical))
            .alias("path_subset"),
        )
        .with_columns(
            pl.col("path_subset")
            .list.reverse()
            .list.eval(pl.element().map_dict(reverse_path_mapping))
            .alias("path_subset_reversed"),
            (
                pl.col("path_subset")
                .list.set_intersection(bc_segments_oriented[0])
                .list.len()
                > 0
            ).alias("is_forward"),
            (
                pl.col("path_subset")
                .list.set_intersection(bc_segments_oriented[1])
                .list.len()
                > 0
            ).alias("is_reverse"),
        )
        .with_columns(
            pl.when(pl.col("is_forward") & pl.col("is_reverse").not_())
            .then(True)
            .when(pl.col("is_forward").not_() & pl.col("is_reverse"))
            .then(pl.lit(False))
            .otherwise(pl.lit(None))
            .alias("orientation"),  # how does this compare to strand?
        )
        .with_columns(
            pl.when(pl.col("orientation") == True)
            .then(pl.col("path_subset"))
            .when(pl.col("orientation") == False)
            .then(pl.col("path_subset_reversed").list.reverse())
            # .otherwise([])
            # .otherwise(pl.col("path_subset_reversed"))
            # pl.when(pl.col("orientation") == True)
            # .then(pl.col("path_subset"))
            # .then(pl.lit(1))
            # .when(pl.col("orientation") == False)
            # .then(pl.col("path_subset_reversed"))
            # .then(pl.lit(2))
            # .otherwise(pl.lit(0))
            # .otherwise(pl.col("path_subset_reversed"))
            .alias("path_subset_normalized")
        )
        .select(pl.col("path_subset_normalized", "read_seq", "read_phred"))
    ).collect()
res

In [ ]:
def consensus_func(df):
    return (
        df.with_columns(
            pl.col("name").str.contains(";").alias("is_duplex"),
            pl.col("name").str.contains(";").not_().alias("is_simplex"),
        )
        .with_columns(
            pl.sum("is_simplex").alias("depth_simplex"),
            pl.sum("is_duplex").alias("depth_duplex"),
        )
        .select(
            pl.col(
                "path_subset_normalized",
                "read_seq",
                "read_phred",
                "depth_simplex",
                "depth_duplex",
            )
        )
        .head(1)
    )

In [ ]:
df.limit(10).with_columns(
    pl.col("name").str.contains(";").alias("is_duplex")
).with_columns(pl.count("is_duplex").alias("num_duplex")).collect()

In [ ]:
%%time
df = pl.scan_pyarrow_dataset(arrow_ds)
with pl.StringCache():
    res = (
        (
            df.with_columns(  # .limit(10000)
                pl.col("path")
                .list.set_intersection(bc_segments)
                .cast(pl.List(pl.Categorical))
                .alias("path_subset"),
            )
            .with_columns(
                pl.col("path_subset")
                .list.reverse()
                .list.eval(pl.element().map_dict(reverse_path_mapping))
                .alias("path_subset_reversed"),
                (
                    pl.col("path")
                    .list.set_intersection(bc_segments_oriented[0])
                    .list.len()
                    > 0
                ).alias("is_forward"),
                (
                    pl.col("path")
                    .list.set_intersection(bc_segments_oriented[1])
                    .list.len()
                    > 0
                ).alias("is_reverse"),
            )
            .with_columns(
                pl.when(pl.col("is_forward") & pl.col("is_reverse").not_())
                .then(True)
                .when(pl.col("is_forward").not_() & pl.col("is_reverse"))
                .then(False)
                .otherwise(None)
                .alias("orientation"),  # how does this compare to strand?
            )
            .with_columns(
                pl.when(pl.col("orientation") == True)
                .then(pl.col("path_subset"))
                .when(pl.col("orientation") == False)
                .then(pl.col("path_subset_reversed").list.reverse())
                .alias("path_subset_normalized")
            )
            .filter(pl.col("path_subset_normalized").hash() % 17 == 3)
            .select(pl.col("path_subset_normalized", "name", "read_seq", "read_phred"))
        )
        .group_by("path_subset_normalized")
        .map_groups(
            consensus_func,
            schema=dict(
                path_subset_normalized=pl.List(pl.Categorical),
                read_seq=pl.Utf8,
                read_phred=pl.Utf8,
                depth_simplex=pl.UInt64,
                depth_duplex=pl.UInt64,
            ),
        )
        .collect()
    )
    # .agg(pl.col("read_seq", "read_phred").map_elements(consensus_func))
res

In [ ]:
df.schema["path"]

In [ ]:
x = res[1]

In [ ]:
x["path_subset_normalized"]

## DuckDB

In [ ]:
%%time
res = duckdb.execute(
    "SELECT path, list_intersect(path, $bc_segments) FROM arrow_ds2 LIMIT 2",
    dict(bc_segments=bc_segments),
).arrow()

In [ ]:
res